# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [53]:
using JuMP, Gurobi
using DataFrames, CSV

In [56]:
# Global path to test system datasets
PATH = joinpath(pwd(), "..", "data", "test_system") 

# Load each of the San Diego test system files 
gens = CSV.read(joinpath(PATH, "gens.csv"), DataFrame);
lines = CSV.read(joinpath(PATH, "lines.csv"), DataFrame);
buses = CSV.read(joinpath(PATH, "buses.csv"), DataFrame);
loads = CSV.read(joinpath(PATH, "loads_24h_sp.csv"), DataFrame);

# Remove Synchrnous Condensers
gens = gens[gens[:, :fueltype] .≠ "Synchronous Condenser", :]

# Add load to bus dataset equal to first hour of load
buses[:, :pd] = loads[:, 2];
println("Consistent buses: ", all(buses[:, :bus] .== loads[:, :bus]))

# Reindex buses go from 1:n
# This is performed to make model indexing consistent 
# with generator and bus indexing.
bus_map = Dict(buses[:, :bus] .=> 1:nrow(buses))
buses = transform(buses, :bus => ByRow(x -> get(bus_map, x, x)) => :bus)
gens = transform(gens, :bus => ByRow(x -> get(bus_map, x, x)) => :bus)
lines = transform(lines, :t_bus => ByRow(x -> get(bus_map, x, x)) => :t_bus)
lines = transform(lines, :f_bus => ByRow(x -> get(bus_map, x, x)) => :f_bus)

# Reindex generators to go from 1:m
gen_map = Dict(gens[:, :id] .=> 1:nrow(gens))
gens = transform(gens, :id => ByRow(x -> get(gen_map, x, x)) => :id)

# Add line susceptance B = X / (R² + X²)
lines[:, :sus] = lines.x ./ (lines.r .^2 + lines.x .^2);

Consistent buses: true


In [74]:
function initialize_model(gens, lines, buses)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """
    # Sets
    G = 1:nrow(gens)       # G: Set of all generators
    N = 1:nrow(buses)      # N: Set of all network nodes
    L = 1:nrow(lines)      # L: Set of all lines/branches
    fbus = lines[!,:f_bus] # All from buses
    tbus = lines[!,:t_bus] # All to buses    
    demand = buses[!,:pd]  # Demand at bus all buses

    # Parameters
    baseMVA = 100      # The base MVA is 100 MVA
    θlim = π*(30/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    shed_cost = 1000   # Cost of load sheding

    # Toy parameters for a limited investment in solar at each bus
    budget = 0
    solar_cost = 0.1475
    
    # Model
    DCOPF = Model(()->Gurobi.Optimizer())
    set_optimizer_attribute(DCOPF, "NumericFocus", 2)
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]          # Generation of each generator 
        SHED[N] ≥ 0     # Load sheading at bus N
        CAP[N]  ≥ 0     # Installed solar capacity at each node
        THETA[N]        # Voltage phase angle of bus
        FLOW[N,N]       # Flows between all pairs of nodes
        FLOW⁺[N,N] ≥ 0  # + Auxiliary flow variable
        FLOW⁻[N,N] ≥ 0  # - Auxiliary flow variable
        LOSS[N,N]  ≥ 0  # Non-negative line loss between a pair of nodes
    end)
                
    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g ∈ G) + sum(shed_cost*SHED[i] for i ∈ N))
    
    # Supply demand balances: sum(generation) + shedding - demand - 0.5*sum(losses) = sum(flows)
    @constraint(DCOPF, cBalance[i ∈ N], 
        sum(GEN[g] for g ∈ G[gens.bus .== i])
        - demand[i] + SHED[i] + CAP[i]
        - 0.5*sum(LOSS[i,j] for j ∈ tbus[fbus .== i])
         == sum(FLOW[i,j] for j ∈ tbus[fbus .== i]))

    # Budget constraint 
    @constraint(DCOPF, cBudget,
        sum(CAP[i] for i ∈ N)*solar_cost ≤ budget)
    
    # Max and min generation constraints
    @constraint(DCOPF, cMaxGen[g ∈ G], GEN[g] ≤ gens[g,:pmax])
    @constraint(DCOPF, cMinGen[g ∈ G], GEN[g] ≥ gens[g,:pmin])
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l ∈ L], 
        FLOW[lines[l,:f_bus], lines[l,:t_bus]] ≤ lines[l,:rate_a]) 

    # Create slack bus with theta=0
    fix(THETA[slack_bus], 0)
    
    # Angle limits 
    @constraint(DCOPF, cAngleLimitsMax[l ∈ L], 
        (THETA[fbus[l]] - THETA[tbus[l]]) ≤  θlim)
    @constraint(DCOPF, cAngleLimitsMin[l ∈ L], 
        (THETA[fbus[l]] - THETA[tbus[l]]) ≥ -θlim)
                    
    # Flow constraints on each branch
    @constraint(DCOPF, cLineFlows[l ∈ L],
            FLOW[fbus[l], tbus[l]] ==
            baseMVA*lines[l,:sus]*(THETA[fbus[l]] - THETA[tbus[l]]))

    # Loss constraints on each line: |FLOWᵢⱼ| = FLOW⁺ᵢⱼ + FLOW⁻ᵢⱼ
    @constraint(DCOPF, cLineLoss[l ∈ L],
            LOSS[fbus[l], tbus[l]] ≥ 
            (lines[l, :r] / baseMVA) * (lines[l, :rate_a])^2
            * (((FLOW⁺[fbus[l], tbus[l]]+FLOW⁻[fbus[l], tbus[l]])
            / lines[l, :rate_a]) - 0.165)
    )

    # Auxiliary flow constraints: FLOWᵢⱼ = FLOW⁺ᵢⱼ - FLOW⁻ᵢⱼ
    @constraint(DCOPF, cAuxFlows[l ∈ L],
            FLOW[fbus[l], tbus[l]] == 
            FLOW⁺[fbus[l], tbus[l]] - FLOW⁻[fbus[l], tbus[l]]
    )
    
    return DCOPF
end

initialize_model (generic function with 1 method)

In [76]:
function model_outputs(model, gens, lines, buses)

    N = 1:nrow(buses)      # N: Set of all network nodes
    fbus = lines[!,:f_bus] # All from buses
    tbus = lines[!,:t_bus] # All to buses
    baseMVA = 100
    
    generation = DataFrame(
        id = gens.id,
        node = gens.bus,
        gen = value.(model[:GEN]).data
        )
    
    # Angles of each bus
    angles = value.(model[:THETA]).data
    
    flows = DataFrame(
        fbus = fbus,
        tbus = tbus,
        flow = baseMVA .* lines.sus .* (angles[fbus] .- angles[tbus])
    )
    
    prices = DataFrame(
        node = N,
        value = dual.(model[:cBalance]).data
    )
    # Amound of load shed at each bus
    shedding = DataFrame(
        node = N,
        shed =value.(model[:SHED]).data
    )
    
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        shedding,
        cost = objective_value(model),
        status = termination_status(model)
    )
end

model_outputs (generic function with 1 method)

In [78]:
dcopf = initialize_model(gens, lines, buses)

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Set parameter NumericFocus to value 2


A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 911675
├ num_constraints: 690864
│ ├ AffExpr in MOI.EqualTo{Float64}: 2673
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2324
│ ├ AffExpr in MOI.LessThan{Float64}: 2325
│ ├ VariableRef in MOI.EqualTo{Float64}: 1
│ └ VariableRef in MOI.GreaterThan{Float64}: 683541
└ Names registered in the model
  └ :CAP, :FLOW, :FLOW⁺, :FLOW⁻, :GEN, :LOSS, :SHED, :THETA, :cAngleLimitsMax, :cAngleLimitsMin, :cAuxFlows, :cBalance, :cBudget, :cLineFlows, :cLineLimits, :cLineLoss, :cMaxGen, :cMinGen

In [80]:
optimize!(dcopf)

Set parameter NumericFocus to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Non-default parameters:
NumericFocus  2

Optimize a model with 7322 rows, 911675 columns and 19355 nonzeros
Model fingerprint: 0x1c36971f
Coefficient statistics:
  Matrix range     [7e-08, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-08, 3e+02]
Presolve removed 4488 rows and 907182 columns
Presolve time: 0.09s
Presolved: 2834 rows, 4633 columns, 11046 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
    3758    2.7004332e+06   0.000000e+00   0.000000e+00      1s

Solved in 3758 iterations and 0.55 seconds (1.36 work units)
Optimal objective  2.700433236e+06

User-callback calls 3895, time in user-callback 0.00 sec


In [82]:
solution = model_outputs(dcopf, gens, lines, buses);

In [84]:
# Get generation by type
gdf = groupby(innerjoin(gens, solution.generation, on = :id), :fueltype)
combine(gdf, :gen => sum)

Row,fueltype,gen_sum
,String,Float64
1,Conventional Hydroelectric,7.75488
2,Other Waste Biomass,0.0
3,Natural Gas Fired Combustion Turbine,70.5192
4,Petroleum Liquids,4.5
5,Landfill Gas,11.8
6,Natural Gas Fired Combined Cycle,34.4568
7,Solar Photovoltaic,58.8121
8,Natural Gas Internal Combustion Engine,3.6
9,Onshore Wind Turbine,4.69156


In [51]:
combine(groupby(gens, :fueltype), :pmax => sum)

Row,fueltype,pmax_sum
,String,Float64
1,Conventional Hydroelectric,8.9
2,Other Waste Biomass,12.3
3,Natural Gas Fired Combustion Turbine,1656.3
4,Petroleum Liquids,4.5
5,Landfill Gas,17.0
6,Natural Gas Fired Combined Cycle,1346.2
7,Solar Photovoltaic,100.848
8,Natural Gas Internal Combustion Engine,3.6
9,Onshore Wind Turbine,194.0


In [43]:
sum(solution.shedding[:, :shed]) / sum(buses[:, :pd])

0.9202622046095534